<a href="https://colab.research.google.com/github/schilds29/PhysChem3/blob/main/workshop3-thermodynamicsB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# B. Acid-Base Pairs
Investigate the gas-phase acidity of __one__ other conjugate acid–base pairs from the list below (see Workshop Instructor for allocation of one of the acid-base pairs below) by calculating the standard Gibbs free energy for acid ionisation at 298.15 K.

* formic acid
* acetic acid
* benzoic acid
* phenol
* methanol




First we need to install the DFT codes (this will take a few minutes)

In [1]:
!pip install -q condacolab

In [1]:
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [2]:
import subprocess
import sys
subprocess.run("rm -rf /usr/local/conda-meta/pinned", shell=True)
subprocess.run("pip -q install py3Dmol", shell=True)
subprocess.run("!mamba install -c anaconda intel-openmp", shell=True)
subprocess.run("conda config --add channels http://conda.anaconda.org/psi4", shell=True)
subprocess.run("mamba install psi4 resp -c conda-forge/label/libint_dev -c conda-forge", shell=True)
subprocess.run("pip install rdkit-pypi", shell=True)
subprocess.run("pip install Cython", shell=True)
subprocess.run("mamba install -c conda-forge parmed -y", shell=True)
subprocess.run("mamba install -c conda-forge openbabel -y", shell=True)
subprocess.run("pip install ase", shell=True)

CompletedProcess(args='pip install ase', returncode=0)

### Instructions
Use the __ase.build.molecule__ routine to create models of the acids and delete one atom to create the congugate base.

In [19]:
from ase.build import molecule
from ase import Atoms

acid = molecule('HCOOH')
print(acid.get_positions())
print(acid.get_atomic_numbers())
print(acid.get_chemical_formula())

base = molecule('HCOOH')
del base[3]
base.set_initial_charges([-1,0,0,0])
print(base.get_positions())
print(base.get_atomic_numbers())
print(base.get_chemical_formula())

molecules = [acid, base]

#code goes here

from ase.io import write
import py3Dmol

write(filename='formic_acid.xyz', images=acid)
view = py3Dmol.view(width=800, height=600)
view.addModel(open('formic_acid.xyz','r').read(),'xyz')
view.addStyle({'stick':{'colorscheme':'greenCarbon','radius':0.3}})
view.setViewStyle({'style':'outline','color':'black','width':0.1})
view.zoomTo()
view.show()


write(filename='formate.xyz', images=base)
view = py3Dmol.view(width=800, height=600)
view.addModel(open('formate.xyz','r').read(),'xyz')
view.addStyle({'stick':{'colorscheme':'greenCarbon','radius':0.3}})
view.setViewStyle({'style':'outline','color':'black','width':0.1})
view.zoomTo()
view.show()

[[-1.040945 -0.436432  0.      ]
 [ 0.        0.423949  0.      ]
 [ 1.169372  0.103741  0.      ]
 [-0.64957  -1.335134  0.      ]
 [-0.377847  1.452967  0.      ]]
[8 6 8 1 1]
CH2O2
[[-1.040945 -0.436432  0.      ]
 [ 0.        0.423949  0.      ]
 [ 1.169372  0.103741  0.      ]
 [-0.377847  1.452967  0.      ]]
[8 6 8 1]
CHO2


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [10]:
#initalise dictionaries to save the results below
acid_results = {}
base_results = {}

Optimise each of the molecules using Psi4 at the _B3LYP/3-21G_ level of theory *with* the BFGS optimiser to maximum force of <0.01 eV/Å.

In [11]:
from ase.calculators.psi4 import Psi4
from ase.build import molecule
import numpy as np
from ase.optimize import QuasiNewton

optimised_molecules = []
for atoms in molecules:
  print(atoms.get_chemical_formula())
  calc = Psi4(atoms = atoms,
        method = 'b3lyp',
        memory = '1000MB', # this is the default, be aware!
        basis = '3-21G',

        charge = np.sum(atoms.get_initial_charges()),
        multiplicity=1,
        label = atoms.get_chemical_formula())

  atoms.calc = calc

  QuasiNewton(atoms).run(fmax=0.01)

  #save results
  if atoms.get_chemical_formula() == 'CH2O2':
      acid_results['energy'] = atoms.get_potential_energy()
  if atoms.get_chemical_formula() == 'CHO2':
      base_results['energy'] = atoms.get_potential_energy()
  print(atoms.get_potential_energy())
  optimised_molecules.append(atoms)

CH2O2
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 05:45:48    -5134.677117       1.3015
BFGSLineSearch:    1[  2] 05:45:59    -5134.700326       0.7495
BFGSLineSearch:    2[  4] 05:46:15    -5134.705147       0.2425
BFGSLineSearch:    3[  6] 05:46:26    -5134.708831       0.1681
BFGSLineSearch:    4[  8] 05:46:37    -5134.711105       0.1733
BFGSLineSearch:    5[ 10] 05:46:48    -5134.713045       0.0967
BFGSLineSearch:    6[ 12] 05:46:59    -5134.713194       0.0234
BFGSLineSearch:    7[ 13] 05:47:05    -5134.713234       0.0076
-5134.7132339434565
CHO2
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 05:47:08    -5118.166967       7.2439
BFGSLineSearch:    1[  2] 05:47:17    -5118.487391       1.8898
BFGSLineSearch:    2[  4] 05:47:26    -5118.529922       0.9779
BFGSLineSearch:    3[  6] 05:47:36    -5118.541681       0.3141
BFGSLineSearch:    4[  8] 05:47:46    -5118.546872       0.1077
BFGSLin

For each of the optimised molecules compute the vibrational and thermodynamic properties using _ase.vibrations_ and _ase.thermochemistry_ codes and save the results to the dictionaries _base\_results_ and _acid\_results_

For example:
```python
from ase.build import molecule
from ase.calculators.emt import EMT
from ase.optimize import QuasiNewton
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo

atoms = molecule('N2')
atoms.calc = Psi4()
potentialenergy = atoms.get_potential_energy()

vib = Vibrations(atoms)
vib.run()
vib_energies = vib.get_energies()

thermo = IdealGasThermo(vib_energies=vib_energies,
                        potentialenergy=potentialenergy,
                        atoms=atoms,
                        geometry='linear',
                        symmetrynumber=2, spin=0)
G = thermo.get_gibbs_energy(temperature=298.15, pressure=101325.)
```

In [21]:
from ase.build import molecule
from ase.calculators.emt import EMT
from ase.optimize import QuasiNewton
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo

atoms = optimised_molecules[0]
atoms.calc = Psi4()
potentialenergy = atoms.get_potential_energy()

vib = Vibrations(atoms)
vib.run()
vib_energies = vib.get_energies()

thermo = IdealGasThermo(vib_energies=vib_energies,
                        potentialenergy=potentialenergy,
                        atoms=atoms,
                        geometry='nonlinear',
                        symmetrynumber=1, spin=1/2)
G = thermo.get_gibbs_energy(temperature=298.15, pressure=101325.)
H = thermo.get_enthalpy(temperature=298.15, verbose=False)
S = thermo.get_entropy(temperature=298.15, pressure=101325., verbose=False)
acid_results['gibbs_energy'] = float(G)
acid_results['enthalpy'] = float(H)
acid_results['entropy'] = float(S)

Enthalpy components at T = 298.15 K:
E_pot              -5138.518 eV
E_ZPE                  0.953 eV
Cv_trans (0->T)        0.039 eV
Cv_rot (0->T)          0.039 eV
Cv_vib (0->T)          0.005 eV
(C_v -> C_p)           0.026 eV
-------------------------------
H                  -5137.457 eV

Entropy components at T = 298.15 K and P = 101325.0 Pa:
                           S               T*S
S_trans (1 bar)    0.0016232 eV/K        0.484 eV
S_rot              0.0009164 eV/K        0.273 eV
S_elec             0.0000597 eV/K        0.018 eV
S_vib              0.0000220 eV/K        0.007 eV
S (1 bar -> P)    -0.0000011 eV/K       -0.000 eV
-------------------------------------------------
S                  0.0026202 eV/K        0.781 eV

Free energy components at T = 298.15 K and P = 101325.0 Pa:
    H      -5137.457 eV
 -T*S         -0.781 eV
-----------------------
    G      -5138.238 eV


In [22]:
from ase.build import molecule
from ase.calculators.emt import EMT
from ase.optimize import QuasiNewton
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo
import numpy as np

atoms = optimised_molecules[1]
atoms.calc = Psi4(charge = np.sum(atoms.get_initial_charges()), multiplicity=1)
potentialenergy = atoms.get_potential_energy()

vib = Vibrations(atoms)
vib.run()
vib_energies = vib.get_energies()

thermo = IdealGasThermo(vib_energies=vib_energies,
                        potentialenergy=potentialenergy,
                        atoms=atoms,
                        geometry='nonlinear',
                        symmetrynumber=1, spin=0)
G = thermo.get_gibbs_energy(temperature=298.15, pressure=101325.)
H = thermo.get_enthalpy(temperature=298.15, verbose=False)
S = thermo.get_entropy(temperature=298.15, pressure=101325., verbose=False)
base_results['gibbs_energy'] = float(G)
base_results['enthalpy'] = float(H)
base_results['entropy'] = float(S)

Enthalpy components at T = 298.15 K:
E_pot              -5123.076 eV
E_ZPE                  0.609 eV
Cv_trans (0->T)        0.039 eV
Cv_rot (0->T)          0.039 eV
Cv_vib (0->T)          0.003 eV
(C_v -> C_p)           0.026 eV
-------------------------------
H                  -5122.361 eV

Entropy components at T = 298.15 K and P = 101325.0 Pa:
                           S               T*S
S_trans (1 bar)    0.0016203 eV/K        0.483 eV
S_rot              0.0008986 eV/K        0.268 eV
S_elec             0.0000000 eV/K        0.000 eV
S_vib              0.0000132 eV/K        0.004 eV
S (1 bar -> P)    -0.0000011 eV/K       -0.000 eV
-------------------------------------------------
S                  0.0025309 eV/K        0.755 eV

Free energy components at T = 298.15 K and P = 101325.0 Pa:
    H      -5122.361 eV
 -T*S         -0.755 eV
-----------------------
    G      -5123.115 eV


In [12]:

from ase.vibrations import Vibrations
#save the results in a list for later processing
vibs = []
for atoms in optimised_molecules:
  print(atoms.get_chemical_formula())
  vib = Vibrations(atoms, name=atoms.get_chemical_formula(), delta=0.01)
  vib.clean()
  vib.run()
  vib.write_mode(0)
  vib.write_mode(-1)
  vib.summary()
  vibs.append(vib)


CH2O2
---------------------
  #    meV     cm^-1
---------------------
  0    1.5i     12.3i
  1    1.3i     10.6i
  2    0.2i      1.4i
  3    0.1       0.7
  4    0.2       1.6
  5    1.9      15.4
  6   75.4     608.2
  7   85.4     688.5
  8  132.0    1064.5
  9  133.1    1073.6
 10  164.7    1328.6
 11  176.8    1425.8
 12  218.9    1765.8
 13  383.1    3089.5
 14  427.7    3449.3
---------------------
Zero-point energy: 0.900 eV
CHO2
---------------------
  #    meV     cm^-1
---------------------
  0    1.0i      8.2i
  1    1.0i      8.1i
  2    0.2i      1.5i
  3    0.2i      1.5i
  4    0.0i      0.2i
  5    3.9      31.3
  6   94.5     762.3
  7  133.0    1072.7
  8  160.7    1296.0
  9  179.8    1450.3
 10  214.8    1732.7
 11  290.9    2345.9
---------------------
Zero-point energy: 0.539 eV


In [27]:
print(acid_results)
print(base_results)

# from part A
h_results = {'energy': 0.0, 'gibbs_energy': -0.27211188035515155, 'enthalpy': 0.0642314260010328, 'entropy': 0.0011281009772134308}

print(h_results)

{'energy': -5134.7132339434565, 'gibbs_energy': -5138.238342459813, 'enthalpy': -5137.457115465838, 'entropy': 0.002620248177007378}
{'energy': -5118.5469505800165, 'gibbs_energy': -5123.1153715459795, 'enthalpy': -5122.360773299954, 'entropy': 0.002530934918750078}
{'energy': 0.0, 'gibbs_energy': -0.27211188035515155, 'enthalpy': 0.0642314260010328, 'entropy': 0.0011281009772134308}


At this point you should have three dictionaries _h\_results_, base\_results_ and _acid\_results_ that includes the electronic energy, enthalpy and entropy of these species.

__Calculate the change in internal energy $\Delta$U, the change in enthalpy $\Delta$H, the entropy contribution T$\Delta$S, and the Gibbs free energy change $\Delta$G for the reaction (Eqns 23-27) and compare to your results for water.__


In [28]:
#answer for deltaH
# deltaH = deltaH(prod) - deltaH(react)

deltaH = (base_results['enthalpy'] + h_results['enthalpy']) - acid_results['enthalpy']
print('deltaH = ' + str(deltaH) + ' ev')

deltaH = 15.160573591884713 ev


In [34]:
#answer for deltaU
# deltaH = detlaU + delta(PV)
# 𝑃𝑉 = kBT

K = 298.15
from ase.units import kB

deltaU = deltaH - (kB * K)
print('deltaU = ' + str(deltaU) + ' eV')

deltaU = 15.1348810214843 eV


In [29]:
#answer for deltaS
# deltaS = deltaS(prod) - deltaS(react)

deltaS = (base_results['entropy'] + h_results['entropy']) - acid_results['entropy']
print('deltaS = ' + str(deltaS) + ' ev/K')

deltaS = 0.001038787718956131 ev/K


In [30]:
#answer for deltaG
# deltaG = deltaG(prod) - delta(react)

deltaG = (base_results['gibbs_energy'] + h_results['gibbs_energy']) - acid_results['gibbs_energy']
print('deltaG = ' + str(deltaG) + ' ev')

deltaG = 14.850859033478628 ev


_Your discussion_

All calculated values are very similar, which makes sense logically as each reaction is similar:

(A) H2O -> H+ + OH-
(B) CH2O2 -> H+ + (CHO2)-

each has 1 reactant (in the forwards direction), and 2 products (hence similar entropy). And the creation/breaking of 1 bond.

__Calculate K$_a$ and pK$_a$ at 298.15 K for the acid-ionisation reaction. Discuss how this compares with the experimentally determined value and why it might differ.__

Experimental values at 298 K can be found in the NIST Chemistry WebBook (https://webbook.nist.gov/chemistry/). In the "Gas phase ion energetics data" section, search under for "De- protonation Reactions" for the acid, or "Gas basicity" for the conjugate base.

In [42]:
#answer for Ka
import math

R = 8.61733e-5
K = 298.15
Ka = math.exp(-(deltaG) / (R * K))
print('Ka = ' + str(Ka) + ' mol/L')

Ka = 9.29878522163571e-252 mol/L


In [43]:
#answer for pKa
# pka = -log(Ka)

import math

pKa = -math.log10(Ka)
print('pKa = ' + str(pKa))

pKa = 251.03157378327396


_Your discussion_

Ka much lower than expected, and hence pKa much higher.
Literature values of: -378.6	kJ/mol

https://webbook.nist.gov/cgi/cbook.cgi?ID=C64186&Units=SI&Mask=1#Thermo-Gas